In [1]:
!pip install pennylane
!pip install qiskit
!pip install qiskit_nature

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyscf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import json
import pennylane as qml
from pennylane import qchem 
import pennylane.numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
from qiskit_nature.drivers import Molecule
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit import Aer
from qiskit.algorithms.optimizers import SLSQP
from qiskit_nature.circuit.library import UCC

import warnings
warnings.filterwarnings('ignore')

<ipython-input-3-345fc2c37443>:12: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
<ipython-input-3-345fc2c37443>:12: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory


# Molecule things

In [4]:
r=1.7
hydrogen_t = [["H", [-r, 0.0, 0.0]], 
              ["Be", [0.0, 0.0, 0.0]], 
              ["H", [r, 0.0, 0.0]]]
                  
h3p = Molecule(
    geometry=  hydrogen_t,
    multiplicity=1, 
    charge=0, 
)

driver = ElectronicStructureMoleculeDriver(h3p, basis="sto-3g", driver_type=ElectronicStructureDriverType.PYSCF) 

properties = driver.run()

In [5]:
num_alpha_electrons = properties.get_property('ParticleNumber').num_alpha
num_beta_electrons = properties.get_property('ParticleNumber').num_beta
num_spin_orbitals = int(properties.get_property('ParticleNumber').num_spin_orbitals)

nuclear_rep_energy = properties.get_property('ElectronicEnergy').nuclear_repulsion_energy
print("number of alpha electrons: " , num_alpha_electrons)
print("number of beta electrons: " , num_beta_electrons)
print("number of spin orbitals: " , num_spin_orbitals)
print("nuclear repulsion energy: " , nuclear_rep_energy)

number of alpha electrons:  3
number of beta electrons:  3
number of spin orbitals:  14
nuclear repulsion energy:  2.6458860546


In [6]:
PN_property = properties.get_property("ParticleNumber")

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=4, #how many electrons we have in our active space
    num_molecular_orbitals=6, #how many orbitals we have in our active space
)

# Now you can get the reduced electronic structure problem
problem_reduced = ElectronicStructureProblem(driver, transformers=[transformer]) 

# The second quantized Hamiltonian of the reduce problem
second_q_ops_reduced = problem_reduced.second_q_ops()


In [7]:
mapper = JordanWignerMapper()

converter = QubitConverter(mapper)

qubit_op = converter.convert(second_q_ops_reduced["ElectronicEnergy"])
print(qubit_op)

-1.0033275559633994 * IIIIIIIIIIII
- 0.22615663558195934 * IIIIIIIIIIIZ
+ 0.04964572261505254 * IIIIIIIYZZZY
+ 0.04964572261505254 * IIIIIIIXZZZX
- 0.23873641581200883 * IIIIIIIIIIZI
+ 0.07746836851665732 * IIIIIIYZZZYI
+ 0.07746836851665732 * IIIIIIXZZZXI
- 0.4339932746983878 * IIIIIIIIIZII
- 0.43399327469838767 * IIIIIIIIZIII
- 0.40499181160592096 * IIIIIIIZIIII
- 0.5805966748013945 * IIIIIIZIIIII
- 0.22615663558195945 * IIIIIZIIIIII
+ 0.049645722615052526 * IYZZZYIIIIII
+ 0.049645722615052526 * IXZZZXIIIIII
- 0.23873641581200888 * IIIIZIIIIIII
+ 0.07746836851665735 * YZZZYIIIIIII
+ 0.07746836851665735 * XZZZXIIIIIII
- 0.4339932746983878 * IIIZIIIIIIII
- 0.43399327469838767 * IIZIIIIIIIII
- 0.40499181160592096 * IZIIIIIIIIII
- 0.5805966748013949 * ZIIIIIIIIIII
+ 0.05434795432072215 * IIIIIIIIIIZZ
- 0.010058511668907727 * IIIIIIYZZZYZ
- 0.010058511668907727 * IIIIIIXZZZXZ
+ 0.0748234314040623 * IIIIIIIIIZIZ
+ 0.0748234314040623 * IIIIIIIIZIIZ
+ 0.06959111091151937 * IIIIIIIZIIIZ
+ 0.0

In [ ]:
vqe_factory = VQEUCCFactory( # This is an example of UCC"SD" ansatz
    quantum_instance=Aer.get_backend("aer_simulator_statevector"),
    optimizer=SLSQP(),
    ansatz=UCC(excitations='sd')
) 

from qiskit.algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

solver = GroundStateEigensolver(converter, vqe_factory)  # Define Numpy
real_solution_t = solver.solve(problem_reduced).total_energies[0]    
print('Reference energy : ', real_solution_t)

In [ ]:
def hydrogen_hamiltonian(coordinates, charge):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.
    
    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(
        [ "H","Be", "H"], coordinates, charge, basis="sto-3g",active_electrons=4, active_orbitals=6
    )[0]

def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.
    
    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns: 
        (int): The number of electrons.
    """
    return 4-charge

def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.
    
    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    # Put your solution here #
    return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

In [ ]:
def BeH2_data(r,threshold, selection=True):
    a0=0.529177210903
    coordinates=[ 0.0, 0.0, -r/a0, 0.0, 0.0, 0.0, 0.0, 0.0, r/a0] # in atomic unit
    charge=0
    hamiltonian = hydrogen_hamiltonian(np.array(coordinates), charge)

    electrons = num_electrons(charge)
    num_qubits = len(hamiltonian.wires)

    hf_state = hf(electrons, num_qubits)
    # singles and doubles are used to make the AllSinglesDoubles template
    singles, doubles = qml.qchem.excitations(electrons, num_qubits)

    dev = qml.device("default.qubit", wires=num_qubits)
    @qml.qnode(dev)
    def circuit_1(params, excitations):
        qml.BasisState(hf_state, wires=range(num_qubits))

        for i, excitation in enumerate(excitations):
            if len(excitation) == 4:
                qml.DoubleExcitation(params[i], wires=excitation)
            else:
                qml.SingleExcitation(params[i], wires=excitation)
        return qml.expval(hamiltonian )

    params = [0.0] * len(doubles)
    params=np.array(params )

    dev = qml.device("default.qubit", wires=num_qubits)
    cost_fn = qml.QNode(circuit_1, dev)
    circuit_gradient = qml.grad(cost_fn, argnum=0)
    grads_d = circuit_gradient(params, excitations=doubles)

    #for i in range(len(doubles)):
    #    print(f"Excitation : {doubles[i%len(doubles)]}, Gradient: {grads_d[i]}")

    params = [0.0] * len(singles)
    params=np.array(params )

    dev = qml.device("default.qubit", wires=num_qubits)
    cost_fn = qml.QNode(circuit_1, dev)
    circuit_gradient = qml.grad(cost_fn, argnum=0)
    grads_s = circuit_gradient(params, excitations=singles)

    #for i in range(len(singles)):
    #    print(f"Excitation : {singles[i%len(singles)]}, Gradient: {grads_s[i]}")

    doubles_select = [(doubles*2)[i] for i in range(len(doubles)) if abs(grads_d[i]) > threshold]
    singles_select = [(singles*2)[i] for i in range(len(singles)) if abs(grads_s[i]) > threshold]

    if selection==True:
        return doubles_select, singles_select, num_qubits, hf_state, hamiltonian
    elif selection==False:
        return doubles, singles, num_qubits, hf_state, hamiltonian

# Original VQE

In [ ]:
def run_VQE(r):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """
    doubles, singles, num_qubits, hf_state, hamiltonian=BeH2_data(r,1e-5, selection=False)
    print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
    print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")
    #print("The original vqe use ",len(doubles + singles)," parameters")
    dev = qml.device("default.qubit", wires=num_qubits)
    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i], wires=doubles[i])

        
        return qml.expval(hamiltonian)
    
    #np.random.seed = 1234
    weights =np.zeros(len(doubles + singles), requires_grad=True)
    #weights = np.random.normal(0, 2*np.pi, len(doubles)+len(singles), requires_grad=True)
    #weights =np.concatenate((params, params_d))
    #opt = qml.GradientDescentOptimizer(stepsize=0.5)
    opt = qml.AdamOptimizer(stepsize=0.05)
    #eta = 0.05
    #opt = qml.QNGOptimizer(0.05)

    i=0
    iter=[]
    cost_val=[]
    Lowest_E=0
    best_weights=0
    for _ in range(100):
        weights = opt.step(cost, weights)
        iter.append(i)
        cost_fn=cost(weights)

        if Lowest_E>cost_fn:
            Lowest_E=cost_fn
            best_weights=weights
        cost_val.append(cost_fn)
        print('iter:',i,' cost_fn:',cost_fn)
        i=i+1

    return Lowest_E, iter, cost_val, best_weights

In [ ]:
Lowest_E, iter, cost_val, best_weights=run_VQE(r)

# Nosiy VQE

In [ ]:
noise_level = 0.1

In [ ]:
def run_VQE_Noisy(r, noise):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """
    doubles, singles, num_qubits, hf_state, hamiltonian=BeH2_data(r,1e-5, selection=False)
    print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
    print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")
    #print("The original vqe use ",len(doubles + singles)," parameters")


    
    # create a bit flip error with probability p = 0.01
    p = noise
    my_bitflip = noise.pauli_error([('X', p), ('I', 1 - p)])

    # create an empty noise model
    my_noise_model = noise.NoiseModel()

    # attach the error to the hadamard gate 'h'
    for i in range(num_qubits):
        my_noise_model.add_quantum_error(my_bitflip, ['measure'], [i])

    dev4 = qml.device('qiskit.aer', wires=num_qubits, noise_model = my_noise_model)

    @qml.qnode(dev4)
    def costNoise(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i], wires=doubles[i])

        
        return qml.expval(hamiltonian)

      
    
    #np.random.seed = 1234
    weights =np.zeros(len(doubles + singles), requires_grad=True)
    #weights = np.random.normal(0, 2*np.pi, len(doubles)+len(singles), requires_grad=True)
    #weights =np.concatenate((params, params_d))
    #opt = qml.GradientDescentOptimizer(stepsize=0.5)
    opt = qml.AdamOptimizer(stepsize=0.05)
    #eta = 0.05
    #opt = qml.QNGOptimizer(0.05)

    i=0
    iter=[]
    cost_val=[]
    Lowest_E=0
    best_weights=0
    for _ in range(100):
        weights = opt.step(cost, weights)
        iter.append(i)
        
        cost_fn=cost(weights)
        probs_NI=np.dot(Minv, probs_fn)
        normalized_probs_NI = probs_NI / np.sqrt(np.sum(probs_NI**2))
        cost_fn=cost(normalized_probs_NI)

        if Lowest_E>cost_fn:
            Lowest_E=cost_fn
            best_weights=weights
        cost_val.append(cost_fn)
        print('iter:',i,' cost_fn:',cost_fn)
        i=i+1

    return Lowest_E, iter, cost_val, best_weights

In [ ]:
Lowest_E_Noisy, iter_Noisy, cost_val_Noisy, best_weights_Noisy=run_VQE_Noisy(r, noise_level)

# Noise Inversion

We can transform noisy data into ideal data. In linear algebra, we do this for a matrix $M$ by finding the inverse matrix $M^{-1}$

$C_{mitigated} = M^{-1} C_{noisy}$

In [ ]:
!pip install bitstring
!pip install pennylane-qiskit

In [ ]:
from bitstring import BitArray
import qiskit
import qiskit.providers.aer.noise as noise


def get_matrix_M(n_qubits, bit_flip_prob):
    length=np.power(2,n_qubits)
    array_of_states=[]
    for i in range(length):
        array_of_states.append(f"{i:0{n_qubits}b}")

    # create a bit flip error with probability p
    p = bit_flip_prob
    my_bitflip = noise.pauli_error([('X', p), ('I', 1 - p)])

    # create an empty noise model
    my_noise_model = noise.NoiseModel()

    # attach the error to the hadamard gate 'h'
    for i in range(n_qubits):
        my_noise_model.add_quantum_error(my_bitflip, ['measure'], [i])

    dev4 = qml.device('qiskit.aer', wires=n_qubits, noise_model = my_noise_model)

    @qml.qnode(dev4)
    def circuit(state):
        for i in range(n_qubits):
            if state[i]=='1':
                qml.PauliX(i)

        return qml.probs()


    M=np.zeros((length, length))
    for state in array_of_states:
        counts = circuit(state)
        print(state+' becomes', counts)

        for st in array_of_states:
            st=int(st,2)
            if counts[st]!=None:
                M[BitArray(bin=state).int][st]=counts[st]
    return M

In [ ]:
M = get_matrix_M(12, noise_level)
import numpy.linalg as la
from qiskit.visualization import array_to_latex

Minv = la.inv(M)

In [ ]:
Minv.shape

In [ ]:
def run_VQE_NI(r, noise):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """
    doubles, singles, num_qubits, hf_state, hamiltonian=BeH2_data(r,1e-5, selection=False)
    print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
    print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")
    #print("The original vqe use ",len(doubles + singles)," parameters")


    
    # create a bit flip error with probability p = 0.01
    p = noise
    my_bitflip = noise.pauli_error([('X', p), ('I', 1 - p)])

    # create an empty noise model
    my_noise_model = noise.NoiseModel()

    # attach the error to the hadamard gate 'h'
    for i in range(num_qubits):
        my_noise_model.add_quantum_error(my_bitflip, ['measure'], [i])

    dev4 = qml.device('qiskit.aer', wires=num_qubits, noise_model = my_noise_model)

    @qml.qnode(dev4)
    def costNoise(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i], wires=doubles[i])

        
        return qml.probs()

      
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def cost(state):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
            
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
                
        """
        qml.MottonenStatePreparation(state_vector=state, wires=list(range(num_qubits)))
            
        return qml.expval(hamiltonian)
    
    #np.random.seed = 1234
    weights =np.zeros(len(doubles + singles), requires_grad=True)
    #weights = np.random.normal(0, 2*np.pi, len(doubles)+len(singles), requires_grad=True)
    #weights =np.concatenate((params, params_d))
    #opt = qml.GradientDescentOptimizer(stepsize=0.5)
    opt = qml.AdamOptimizer(stepsize=0.05)
    #eta = 0.05
    #opt = qml.QNGOptimizer(0.05)

    i=0
    iter=[]
    cost_val=[]
    Lowest_E=0
    best_weights=0
    for _ in range(100):
        weights = opt.step(cost, weights)
        iter.append(i)
        
        probs_fn=cost(weights)
        probs_NI=np.dot(Minv, probs_fn)
        normalized_probs_NI = probs_NI / np.sqrt(np.sum(probs_NI**2))
        cost_fn=cost(normalized_probs_NI)

        if Lowest_E>cost_fn:
            Lowest_E=cost_fn
            best_weights=weights
        cost_val.append(cost_fn)
        print('iter:',i,' cost_fn:',cost_fn)
        i=i+1

    return Lowest_E, iter, cost_val, best_weights

In [ ]:
Lowest_E_NI, iter_NI, cost_val_NI, best_weights_NI=run_VQE_NI(r, noise_level)

# VQE with TFLO

In [ ]:
def run_VQE_FLO(r):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """
    doubles, singles, num_qubits, hf_state, hamiltonian=BeH2_data(r,1e-5, selection=False)
    print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
    print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")
    #print("The original vqe use ",len(doubles + singles)," parameters")
    dev = qml.device("default.qubit", wires=num_qubits)
    @qml.qnode(dev)
    def cost(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(0, wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i], wires=doubles[i])

        
        return qml.expval(hamiltonian)
    
    #np.random.seed = 1234
    weights =np.zeros(len(doubles + singles), requires_grad=True)
    #weights = np.random.normal(0, 2*np.pi, len(doubles)+len(singles), requires_grad=True)
    #weights =np.concatenate((params, params_d))
    #opt = qml.GradientDescentOptimizer(stepsize=0.5)
    opt = qml.AdamOptimizer(stepsize=0.05)
    #eta = 0.05
    #opt = qml.QNGOptimizer(0.05)

    i=0
    iter=[]
    cost_val=[]
    Lowest_E=0
    best_weights=0
    for _ in range(100):
        weights = opt.step(cost, weights)
        iter.append(i)
        cost_fn=cost(weights)

        if Lowest_E>cost_fn:
            Lowest_E=cost_fn
            best_weights=weights
        cost_val.append(cost_fn)
        print('iter:',i,' cost_fn:',cost_fn)
        i=i+1

    return Lowest_E, iter, cost_val, best_weights

In [ ]:
Lowest_E_FLO, iter_FLO, cost_val_FLO, best_weights_FLO=run_VQE_FLO(r)

In [ ]:
def run_VQE_Noisy_FLO(r, noise):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """
    doubles, singles, num_qubits, hf_state, hamiltonian=BeH2_data(r,1e-5, selection=False)
    print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
    print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")
    #print("The original vqe use ",len(doubles + singles)," parameters")


    
    # create a bit flip error with probability p = 0.01
    p = noise
    my_bitflip = noise.pauli_error([('X', p), ('I', 1 - p)])

    # create an empty noise model
    my_noise_model = noise.NoiseModel()

    # attach the error to the hadamard gate 'h'
    for i in range(num_qubits):
        my_noise_model.add_quantum_error(my_bitflip, ['measure'], [i])

    dev4 = qml.device('qiskit.aer', wires=num_qubits, noise_model = my_noise_model)

    @qml.qnode(dev4)
    def costNoise(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(0, wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i], wires=doubles[i])

        
        return qml.expval(hamiltonian)

      
    
    #np.random.seed = 1234
    weights =np.zeros(len(doubles + singles), requires_grad=True)
    #weights = np.random.normal(0, 2*np.pi, len(doubles)+len(singles), requires_grad=True)
    #weights =np.concatenate((params, params_d))
    #opt = qml.GradientDescentOptimizer(stepsize=0.5)
    opt = qml.AdamOptimizer(stepsize=0.05)
    #eta = 0.05
    #opt = qml.QNGOptimizer(0.05)

    i=0
    iter=[]
    cost_val=[]
    Lowest_E=0
    best_weights=0
    for _ in range(100):
        weights = opt.step(cost, weights)
        iter.append(i)
        
        cost_fn=cost(weights)
        probs_NI=np.dot(Minv, probs_fn)
        normalized_probs_NI = probs_NI / np.sqrt(np.sum(probs_NI**2))
        cost_fn=cost(normalized_probs_NI)

        if Lowest_E>cost_fn:
            Lowest_E=cost_fn
            best_weights=weights
        cost_val.append(cost_fn)
        print('iter:',i,' cost_fn:',cost_fn)
        i=i+1

    return Lowest_E, iter, cost_val, best_weights

In [ ]:
Lowest_E_NoisyFLO, iter_NoisyFLO, cost_val_NoisyFLO, best_weights_NoisyFLO=run_VQE_Noisy_FLO(r, noise_level)

In [ ]:
Lowest_E_TFLO = Lowest_E_Noisy + Lowest_E_FLO - Lowest_E_NoisyFLO

# Noise Inversion + TFLO

In [ ]:
def run_VQE_NI_FLO(r, noise):
    """Performs a VQE routine for the given hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.:

    Returns:
        (float): The expectation value of the hydrogen Hamiltonian.
    """
    doubles, singles, num_qubits, hf_state, hamiltonian=BeH2_data(r,1e-5, selection=False)
    print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
    print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")
    #print("The original vqe use ",len(doubles + singles)," parameters")


    
    # create a bit flip error with probability p = 0.01
    p = noise
    my_bitflip = noise.pauli_error([('X', p), ('I', 1 - p)])

    # create an empty noise model
    my_noise_model = noise.NoiseModel()

    # attach the error to the hadamard gate 'h'
    for i in range(num_qubits):
        my_noise_model.add_quantum_error(my_bitflip, ['measure'], [i])

    dev4 = qml.device('qiskit.aer', wires=num_qubits, noise_model = my_noise_model)

    @qml.qnode(dev4)
    def costNoise(weights):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
        
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
            
        """
        #state=np.array([0]*(num_qubits//2)+[1]*(num_qubits//2))
        qml.BasisState(hf_state, wires=list(range(num_qubits)))

        for i in range(len(singles)):
            #for j in range(E_len):
            qml.SingleExcitation(0, wires=singles[i])

        for i in range(len(doubles)):
            qml.DoubleExcitation(weights[i], wires=doubles[i])

        
        return qml.probs()

      
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def cost(state):
        """A circuit with tunable parameters/weights that measures the expectation value of the hydrogen Hamiltonian.
            
        Args:
            weights (numpy.array): An array of tunable parameters.

        Returns:
            (float): The expectation value of the hydrogen Hamiltonian.
                
        """
        qml.MottonenStatePreparation(state_vector=state, wires=list(range(num_qubits)))
            
        return qml.expval(hamiltonian)
    
    #np.random.seed = 1234
    weights =np.zeros(len(doubles + singles), requires_grad=True)
    #weights = np.random.normal(0, 2*np.pi, len(doubles)+len(singles), requires_grad=True)
    #weights =np.concatenate((params, params_d))
    #opt = qml.GradientDescentOptimizer(stepsize=0.5)
    opt = qml.AdamOptimizer(stepsize=0.05)
    #eta = 0.05
    #opt = qml.QNGOptimizer(0.05)

    i=0
    iter=[]
    cost_val=[]
    Lowest_E=0
    best_weights=0
    for _ in range(100):
        weights = opt.step(cost, weights)
        iter.append(i)
        
        probs_fn=cost(weights)
        probs_NI=np.dot(Minv, probs_fn)
        normalized_probs_NI = probs_NI / np.sqrt(np.sum(probs_NI**2))
        cost_fn=cost(normalized_probs_NI)

        if Lowest_E>cost_fn:
            Lowest_E=cost_fn
            best_weights=weights
        cost_val.append(cost_fn)
        print('iter:',i,' cost_fn:',cost_fn)
        i=i+1

    return Lowest_E, iter, cost_val, best_weights

In [ ]:
Lowest_E_NIFLO, iter_NIFLO, cost_val_NIFLO, best_weights_NIFLO=run_VQE_NI_FLO(r, noise_level)

In [ ]:
Lowest_E_NITFLO = Lowest_E_Noisy + Lowest_E_FLO - Lowest_E_NIFLO

# Results

In [ ]:
print(f"energy VQE = {Lowest_E}")
print(f"energy VQE with noise = {Lowest_E_Noisy}")
print(f"energy VQE with NI = {Lowest_E_NI}")
print(f"energy VQE with TFLO = {Lowest_E_TFLO}")
print(f"energy VQE with NI and TFLO = {Lowest_E_NITFLO}")